# Advanced Production Planning Problem

In [1]:
!pip install pulp
from pulp import *
import pandas as pd

In [2]:
# Creates a list of the products
months = ['000','JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']

# A dictionary of the unit cost per month
cost = {'000': 0,
        'JAN': 240, 
        'FEB': 250,
        'MAR': 265, 
        'APR': 285,
        'MAY': 280, 
        'JUN': 260}

# A dictionary of the demand per month
demand = {'000': 0,
          'JAN': 1000, 
          'FEB': 4500,
          'MAR': 6000, 
          'APR': 5500,
          'MAY': 3500, 
          'JUN': 4000}

# A dictionary of the min production per month
minOut = {'000': 0,
          'JAN': 2000, 
          'FEB': 1750,
          'MAR': 2000, 
          'APR': 2250,
          'MAY': 2000, 
          'JUN': 1750}

# A dictionary of the max production per month
maxOut = {'JAN': 4000, 
          'FEB': 3500,
          'MAR': 4000, 
          'APR': 4500,
          'MAY': 4000, 
          'JUN': 3500,
          '000': 0}


In [3]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Upton_Corporation_Production", LpMinimize)

# A dictionary of the output variables
o = LpVariable.dicts("output",months,0)

# A dictionary of the storage variables
# with safety stock and maximum warehoues capacity as bounds
s = LpVariable.dicts("storage",months,1500,6000)

# Create objective function
prob += lpSum([o[i]*cost[i] for i in months]) + lpSum([0.015*cost[months[i+1]]*0.5*(s[months[i+1]]+s[months[i]]) for i in range(6)]), "Total Cost"

#Enforce values of init period
prob += s["000"] == 2750

# Create constraints
for month in months:
    prob += o[month] >= minOut[month], "minOut{}".format(month)
    prob += o[month] <= maxOut[month], "maxOut{}".format(month)

for i in range(6):
    prob += s[months[i]]+o[months[i+1]]-demand[months[i+1]]==s[months[i+1]]

In [4]:
# Optimize

prob.solve()

# Print the status of the solved LP
print("Status = %s" % LpStatus[prob.status])

Status = Optimal


In [5]:
# Print the value of the variables at the optimum
output = pd.DataFrame([[i, s[i].varValue, o[i].varValue] for i in months],
            columns = ['month', 's', 'o']).set_index('month')
print(output)

# Print the value of the objective
print("Objective = %.1f" % value(prob.objective))

            s       o
month                
000    2750.0     0.0
JAN    5750.0  4000.0
FEB    4750.0  3500.0
MAR    2750.0  4000.0
APR    1500.0  4250.0
MAY    2000.0  4000.0
JUN    1500.0  3500.0
Objective = 6209403.1
